In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.graph_objects as go 

import torch 
from torch import nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

from typing import Optional,Callable


In [20]:
X,y,coef = datasets.make_regression(n_features=2,n_samples=20,coef=True,noise=2,random_state=1,bias=20)

# label_scaler = StandardScaler()
# y = label_scaler.fit_transform(y.reshape(-1,1))


# print(coef)

In [21]:
print(X.shape)
print(y.shape)

(20, 2)
(20,)


In [22]:
def plot_data(X1:torch.Tensor,
              y:torch.Tensor,
              X2:Optional[torch.Tensor]= None,
              bias:Optional[torch.Tensor]= None,
              n_samples = 300):
  
  if not isinstance(X1,torch.Tensor):
    X1 = torch.tensor(X1)
  X1 = X1.flatten()
  

  #-------------------------------------ONE FEATURES-----------------------------------
  if (X2 is None) and (bias is None):
    plot = go.Scatter(
      x = X1,
      y = y,
      mode = 'markers',
    )


  #-------------------------------------TWO FEATURES-----------------------------------
  if (X2 is not None) and (bias is None):
    plot = go.Scatter3d(
      x = X1,
      y = X2,
      z = y,
      mode = 'markers'
    )
    

  return plot



In [23]:
plot = plot_data(X[:,0],X[:,1],y)

figure = go.Figure(data=[plot])
figure.show()

In [5]:
def landscape(X1:torch.Tensor,
              y:torch.Tensor,
              X2:Optional[torch.Tensor]= None,
              bias = False,
              cost_fn = nn.MSELoss(),
              span = 10,
              n_samples = 300):

  # Type & size
  if not isinstance(X1,torch.Tensor):
    X1 = torch.tensor(X1)
  X1 = X1.view(-1,1)
  if not isinstance(y,torch.Tensor):
    y = torch.tensor(y)
  y = y.view(-1,1).float()


  cost_fn = cost_fn

  #-------------------------------------ONE FEATURES-----------------------------------
  if (X2 is None) and (bias is False):
    landscape = go.Scatter(
      x = torch.linspace(-span,span,n_samples),
      y = [cost_fn((X1*w),y).item() for w in torch.linspace(-span,span,n_samples)],
      mode = 'lines'
    )
  
  #-------------------------------------TWO FEATURES-----------------------------------
  if (X2 is not None) and (bias is False):
    if not isinstance(X2,torch.Tensor):
      X2 = torch.tensor(X2)
    X2 = X2.view(-1,1)


    w1_m,w2_m = torch.meshgrid(torch.linspace(-span,span,n_samples),torch.linspace(-span,span,n_samples),indexing='ij')
    w1_f,w2_f = w1_m.flatten() , w2_m.flatten()

    COST = []
    for w1,w2 in zip(w1_f,w2_f):
      pred = (w1*X1) + (w2*X2)
      cost = cost_fn(pred,y)
      COST.append(cost.item())
    COST = torch.tensor(COST).view(n_samples,n_samples)

    landscape = go.Surface(
      x = w1_m,
      y = w2_m,
      z = COST,
    )

  # -------------------------------------ONE FEATURE AND BIAS-----------------------------------
  if (X2 is  None) and (bias is True):


    w1_m,b_m = torch.meshgrid(torch.linspace(-span,span,n_samples),torch.linspace(-span,span,n_samples),indexing='ij')
    w1_f,b_f = w1_m.flatten() , b_m.flatten()

    COST = []
    for w1,b in zip(w1_f,b_f):
      pred = (w1*X1) + b
      cost = cost_fn(pred,y)
      COST.append(cost.item())
    COST = torch.tensor(COST).view(n_samples,n_samples)

    landscape = go.Surface(
      x = w1_m,
      y = b_m,
      z = COST,
    )


  return landscape

In [8]:
Landscape = landscape(X1 = X[:,0],
                      # X2 = X[:,1], 
                      y=y,
                      bias = True,
                      cost_fn=nn.L1Loss(),
                      n_samples=50,
                      span=200
                      )

figure = go.Figure(data=[Landscape])
figure.show()

In [17]:
a = torch.randint(0,10,(20,2))
b = torch.randint(0,10,(20,))


graph = go.Scatter3d(
  x = a[:,0],
  y = a[:,1],
  z = b,
  mode = 'markers'
)

figure = go.Figure(data=[graph])
figure.show()